In [11]:
import os
import pandas
from functools import partial, reduce

## Equation

$$ DCF = \frac{CF_1}{(1 + r) ^ 1} + \frac{CF_2}{(1 + r) ^ 2} + ... + \frac{CF_n}{(1 + r) ^ n} $$

Where CF stands for unlevered free cash flow for a given year ($CF_1$ means cash flow in year 1, and so on); and $r$ stands for the discount rate.

CF = EBITDA - capital expenditure - changes working capital - taxes

## Load Data

In [4]:
sample_path = os.getcwd() + '/sample-output/'

In [6]:
file_name = [x for x in os.listdir('sample-output/') if x.endswith('.csv')]

In [7]:
dfs = []
for f in file_name:
    dfs.append(pandas.read_csv(sample_path + f))

In [12]:
df = reduce(partial(pandas.merge, on = 'ticker_code'), dfs)

## Calculate CF

In [17]:
# cf = pretax_income - fixed_assets - (total_current_assets - total_current_liabilities)

In [18]:
df_cf = df[['ticker_code', 'pretax_income', 'fixed_assets', 'total_current_assets', 'total_current_liabilities']]

In [23]:
def get_free_cash_flow(data):
    
    return data[['pretax_income' - 'fixed_assets' - 'total_current_assets' - 'total_current_liabilities']]

In [26]:
df_cf['free_cash_flow'] = df['pretax_income'] - df['fixed_assets']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df_cf.head(10)

,ticker_code,pretax_income,fixed_assets,total_current_assets,total_current_liabilities,free_cash_flow
0,AALI,2.207080e+06,1.021901e+07,4.500628e+06,3.076530e+06,-8.011933e+06
1,ABBA,-4.132766e+10,3.423789e+10,1.310550e+11,1.880980e+11,-7.556555e+10
2,ABDA,5.082946e+07,9.637895e+07,NaN,NaN,-4.554948e+07
3,ABMM,9.362569e+07,4.010247e+08,3.252425e+08,2.227963e+08,-3.073990e+08
4,ACES,1.202709e+12,4.438951e+11,4.096280e+12,6.310555e+11,7.588140e+11
5,ACST,2.422600e+04,7.551290e+05,NaN,NaN,-7.309030e+05
6,ADES,7.006000e+04,4.472490e+05,3.641380e+05,2.623970e+05,-3.771890e+05
7,ADHI,1.167777e+12,1.573324e+12,2.542954e+13,1.896430e+13,-4.055467e+11
8,ADMF,2.484569e+06,2.454880e+05,NaN,NaN,2.239081e+06
9,ADMG,-1.395757e+06,1.475900e+08,1.241804e+08,2.646348e+07,-1.489857e+08
